To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Introducing FP8 precision training for faster RL inference. [Read Blog](https://docs.unsloth.ai/new/fp8-reinforcement-learning).

Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!git clone https://github.com/SparkAudio/Spark-TTS
!pip install omegaconf einx torchcodec "datasets>=3.4.1,<4.0.0"

In [2]:
!git clone https://github.com/SparkAudio/Spark-TTS
!pip install omegaconf einx torchcodec "datasets>=3.4.1,<4.0.0"

fatal: destination path 'Spark-TTS' already exists and is not an empty directory.


### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

Thank you to [Etherl](https://huggingface.co/Etherll) for creating this notebook!

In [3]:
!uv pip install torchvision>=0.24.0 --system 

Using Python 3.11.7 environment at: /usr
Resolved 28 packages in 62ms                                         
Uninstalled 2 packages in 139ms
Installed 2 packages in 209ms                               
 - torch==2.9.0
 + torch==2.9.1
 - triton==3.5.0
 + triton==3.5.1


In [4]:
!uv pip install wandb --upgrade --system

Using Python 3.11.7 environment at: /usr
Resolved 20 packages in 92ms                                         
Audited 20 packages in 0.34ms


In [5]:
from unsloth import FastModel
import torch
from huggingface_hub import snapshot_download

max_seq_length = 2048 # Choose any for long context!

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
    # Qwen3 new models
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # Other very popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

# Download model and code
snapshot_download("SparkAudio/Spark-TTS-0.5B", local_dir = "Spark-TTS-0.5B")

model, tokenizer = FastModel.from_pretrained(
    model_name = "Spark-TTS-0.5B/LLM",
    max_seq_length = max_seq_length,
    dtype = torch.float32, # Spark seems to only work on float32 for now
    full_finetuning = True, # We support full finetuning now!
    load_in_4bit = False,
    #token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-01-11 18:38:02.440131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-11 18:38:02.440257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-11 18:38:02.519316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-11 18:38:02.682852: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-11 18:38:04.204953: W tensorflow/compiler/tf2

🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.alignprop_trainer: Failed to import trl.trainer.alignprop_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)
Unsloth: Could not import trl.trainer.ddpo_trainer: Failed to import trl.trainer.ddpo_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
cannot import name 'cached_download' from 'huggingface_hub' (/usr/local/lib/python3.11/dist-packages/huggingface_hub/__init__.py)
==((====))==  Unsloth 2026.1.2: Fast Qwen2 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [6]:
#LoRA does not work with float32 only works with bfloat16 !!!
model = FastModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Full finetuning is enabled, so .get_peft_model has no effect


In [ ]:
!huggingface-cli login --token {HF_TOKEN}
!wandb login {WANDB_TOKEN}

<a name="Data"></a>
### Data Prep  

We will use the `MrDragonFox/Elise`, which is designed for training TTS models. Ensure that your dataset follows the required format: **text, audio** for single-speaker models or **source, text, audio** for multi-speaker models. You can modify this section to accommodate your own dataset, but maintaining the correct structure is essential for optimal training.

In [7]:
import re
from typing import Callable, Tuple

# ----------------------------
# Core lexicon (Egyptian-ish)
# ----------------------------

_AR_DIGITS = {
    0: "صفر", 1: "واحد", 2: "اتنين", 3: "تلاتة", 4: "أربعة", 5: "خمسة",
    6: "ستة", 7: "سبعة", 8: "تمانية", 9: "تسعة", 10: "عشرة",
    11: "حداشر", 12: "اتناشر", 13: "تلتاشر", 14: "أربعتاشر", 15: "خمستاشر",
    16: "ستاشر", 17: "سبعتاشر", 18: "تمانتاشر", 19: "تسعتاشر"
}

_AR_TENS = {
    20: "عشرين", 30: "تلاتين", 40: "أربعين", 50: "خمسين",
    60: "ستين", 70: "سبعين", 80: "تمانين", 90: "تسعين"
}

_AR_HUNDREDS = {
    100: "مية", 200: "ميتين", 300: "تلتمية", 400: "أربعمية",
    500: "خمسمية", 600: "ستمية", 700: "سبعمية",
    800: "تمنمية", 900: "تسعمية"
}

_SCALES = [
    (1_000_000_000, "مليار", "مليارين", "ملايير"),
    (1_000_000, "مليون", "مليونين", "ملايين"),
    (1_000, "ألف", "ألفين", "آلاف"),
]

# Arabic digit mappings
_ARABIC_TO_WESTERN = str.maketrans("٠١٢٣٤٥٦٧٨٩", "0123456789")

DECIMAL_WORD = "نقطة"
PERCENT_WORD = "في المية"

# Currency words with proper pluralization
CURRENCY = {
    "EGP": ("جنيه", "جنيه", "جنيه", "قرش", "قرشين", "قروش"),
    "ج": ("جنيه", "جنيه", "جنيه", "قرش", "قرشين", "قروش"),
    "جنيه": ("جنيه", "جنيجنيههين", "جنيه", "قرش", "قرشين", "قروش"),
    "LE": ("جنيه", "جنيه", "جنيه", "قرش", "قرشين", "قروش"),
    "£": ("جنيه", "جنيه", "جنيه", "قرش", "قرشين", "قروش"),
    "$": ("دولار", "دولارين", "دولار", "سنت", "سنتين", "سنتات"),
    "USD": ("دولار", "دولارين", "دولار", "سنت", "سنتين", "سنتات"),
    "€": ("يورو", "يوروهين", "يوروهات", "سنت", "سنتين", "سنتات"),
    "EUR": ("يورو", "يوروهين", "يوروهات", "سنت", "سنتين", "سنتات"),
}

# Ordinal numbers (for dates, floors, etc.)
_AR_ORDINALS = {
    1: "الأول", 2: "التاني", 3: "التالت", 4: "الرابع", 5: "الخامس",
    6: "السادس", 7: "السابع", 8: "التامن", 9: "التاسع", 10: "العاشر",
    11: "الحداشر", 12: "الاتناشر"
}

# Month names
_AR_MONTHS = {
    1: "يناير", 2: "فبراير", 3: "مارس", 4: "أبريل", 5: "مايو", 6: "يونيو",
    7: "يوليو", 8: "أغسطس", 9: "سبتمبر", 10: "أكتوبر", 11: "نوفمبر", 12: "ديسمبر"
}


# ----------------------------
# Helper functions
# ----------------------------

def normalize_arabic_digits(text: str) -> str:
    """Convert Arabic-Indic digits to Western digits."""
    return text.translate(_ARABIC_TO_WESTERN)


def get_plural_form(n: int, singular: str, dual: str, plural: str) -> str:
    """Return appropriate Arabic plural form based on number."""
    if n == 1:
        return singular
    elif n == 2:
        return dual
    else:
        return plural


# ----------------------------
# Number to Egyptian Arabic-ish
# ----------------------------

def int_to_egyptian_words(n: int) -> str:
    """Convert integer (>=0) into Egyptian Arabic-ish words with better scaling."""
    if n < 0:
        return "سالب " + int_to_egyptian_words(-n)
    if n < 20:
        return _AR_DIGITS[n]
    if n < 100:
        tens = (n // 10) * 10
        ones = n % 10
        if ones == 0:
            return _AR_TENS[tens]
        return f"{_AR_DIGITS[ones]} و{_AR_TENS[tens]}"
    if n < 1000:
        hundreds = (n // 100) * 100
        rest = n % 100
        if rest == 0:
            return _AR_HUNDREDS[hundreds]
        return f"{_AR_HUNDREDS[hundreds]} و{int_to_egyptian_words(rest)}"

    for scale_value, scale_singular, scale_dual, scale_plural in _SCALES:
        if n >= scale_value:
            major = n // scale_value
            rest = n % scale_value
            
            # Better pluralization
            if major == 1:
                major_words = scale_singular
            elif major == 2:
                major_words = scale_dual
            elif major <= 10:
                major_words = f"{int_to_egyptian_words(major)} {scale_plural}"
            else:
                major_words = f"{int_to_egyptian_words(major)} {scale_singular}"
            
            if rest:
                major_words += f" و{int_to_egyptian_words(rest)}"
            return major_words

    return str(n)


def num_token_to_words(token: str) -> str:
    """
    Convert token like:
      - "123" -> words
      - "12.5" or "12,5" -> words with DECIMAL_WORD
    Handles Arabic-Indic digits as well.
    """
    # Normalize Arabic digits first
    t = normalize_arabic_digits(token)
    t = t.replace("٬", "").replace(",", ".")
    
    if not re.fullmatch(r"-?\d+(\.\d+)?", t):
        return token

    neg = t.startswith("-")
    if neg:
        t = t[1:]

    if "." in t:
        a, b = t.split(".", 1)
        a_words = int_to_egyptian_words(int(a)) if a else "صفر"
        # Speak each digit in decimal part
        b_words = " ".join(_AR_DIGITS[int(ch)] for ch in b if ch.isdigit())
        out = f"{a_words} {DECIMAL_WORD} {b_words}".strip()
    else:
        out = int_to_egyptian_words(int(t))

    return ("سالب " + out) if neg else out


# ----------------------------
# Specific pattern normalizers
# ----------------------------

def normalize_percent(text: str) -> str:
    """Handle percentages including Arabic digits and symbols."""
    def repl(m):
        num = m.group("num")
        return f"{num_token_to_words(num)} {PERCENT_WORD}"
    
    # Handle both % and ٪ (Arabic percent)
    return re.sub(r"(?P<num>-?[\d٠-٩]+(?:[.,٫][\d٠-٩]+)?)\s*[%٪]+", repl, text)


def normalize_currency(text: str) -> str:
    """
    Enhanced currency handling with proper pluralization.
    Handles: "100 EGP", "100ج", "$12.50", "€30"
    """
    def sym_first(m):
        sym = m.group("sym")
        num = m.group("num")
        
        if sym not in CURRENCY:
            return m.group(0)
        
        major_sg, major_du, major_pl, minor_sg, minor_du, minor_pl = CURRENCY[sym]
        
        # Normalize and parse
        t = normalize_arabic_digits(num).replace(",", ".")
        
        if "." in t:
            a, b = t.split(".", 1)
            a_i = int(a) if a else 0
            b2 = (b + "00")[:2]
            b_i = int(b2)
            
            major_word = get_plural_form(a_i, major_sg, major_du, major_pl)
            
            if b_i == 0:
                return f"{int_to_egyptian_words(a_i)} {major_word}"
            
            minor_word = get_plural_form(b_i, minor_sg, minor_du, minor_pl)
            return f"{int_to_egyptian_words(a_i)} {major_word} و{int_to_egyptian_words(b_i)} {minor_word}"
        
        a_i = int(t)
        major_word = get_plural_form(a_i, major_sg, major_du, major_pl)
        return f"{num_token_to_words(num)} {major_word}"

    text = re.sub(r"(?P<sym>[$€£])\s*(?P<num>-?[\d٠-٩]+(?:[.,٫][\d٠-٩]+)?)", sym_first, text)

    # Num-first pattern
    def num_first(m):
        num = m.group("num")
        cur = m.group("cur").strip()
        
        if cur not in CURRENCY:
            return m.group(0)
        
        major_sg, major_du, major_pl = CURRENCY[cur][:3]
        num_normalized = normalize_arabic_digits(num).replace(",", ".")
        
        if "." not in num_normalized:
            n = int(num_normalized)
            major_word = get_plural_form(n, major_sg, major_du, major_pl)
        else:
            major_word = major_sg
        
        return f"{num_token_to_words(num)} {major_word}"

    return re.sub(r"(?P<num>-?[\d٠-٩]+(?:[.,٫][\d٠-٩]+)?)\s*(?P<cur>EGP|USD|EUR|LE|جنيه|ج)", num_first, text)


def normalize_dates(text: str) -> str:
    """
    Handle date patterns:
    - 15/3/2024 -> "خمستاشر مارس ألفين وأربعة وعشرين"
    - 2024-03-15
    """
    def repl_slash(m):
        day = int(normalize_arabic_digits(m.group("day")))
        month = int(normalize_arabic_digits(m.group("month")))
        year = int(normalize_arabic_digits(m.group("year"))) if m.group("year") else None
        
        day_word = _AR_ORDINALS.get(day, int_to_egyptian_words(day))
        month_word = _AR_MONTHS.get(month, int_to_egyptian_words(month))
        
        if year:
            year_word = int_to_egyptian_words(year)
            return f"{day_word} {month_word} {year_word}"
        return f"{day_word} {month_word}"
    
    # DD/MM/YYYY or DD/MM
    text = re.sub(
        r"\b(?P<day>[\d٠-٩]{1,2})[/\-](?P<month>[\d٠-٩]{1,2})(?:[/\-](?P<year>[\d٠-٩]{2,4}))?\b",
        repl_slash,
        text
    )
    
    return text


def normalize_time(text: str) -> str:
    """Enhanced time handling with more natural Egyptian expressions."""
    def repl(m):
        hh = int(normalize_arabic_digits(m.group("hh")))
        mm = int(normalize_arabic_digits(m.group("mm")))
        
        h_words = int_to_egyptian_words(hh)
        
        if mm == 0:
            return f"{h_words} بالظبط"
        if mm == 15:
            return f"{h_words} وربع"
        if mm == 30:
            return f"{h_words} ونص"
        if mm == 45:
            next_h = (hh + 1) % 24
            return f"{int_to_egyptian_words(next_h)} إلا ربع"
        
        # For other minutes
        m_words = int_to_egyptian_words(mm)
        if mm == 1:
            return f"{h_words} ودقيقة"
        elif mm == 2:
            return f"{h_words} ودقيقتين"
        else:
            return f"{h_words} و{m_words} دقيقة"
    
    return re.sub(r"\b(?P<hh>[0-2]?[\d٠-٩]):(?P<mm>[0-5][\d٠-٩])\b", repl, text)


def normalize_ranges(text: str) -> str:
    """Handle numeric ranges with better context awareness."""
    def repl(m):
        a, b = m.group("a"), m.group("b")
        return f"من {num_token_to_words(a)} لحد {num_token_to_words(b)}"
    
    return re.sub(
        r"\b(?P<a>-?[\d٠-٩]+(?:[.,٫][\d٠-٩]+)?)\s*[-–—]\s*(?P<b>-?[\d٠-٩]+(?:[.,٫][\d٠-٩]+)?)\b",
        repl,
        text
    )


def normalize_phone_like(text: str) -> str:
    """
    Enhanced phone number handling.
    Recognizes Egyptian phone patterns (01X XXXX XXXX).
    """
    def repl(m):
        s = normalize_arabic_digits(m.group(0))
        s = re.sub(r"\D", "", s)
        
        # Don't process if too short (likely not a phone number)
        if len(s) < 7:
            return m.group(0)
        
        return " ".join(_AR_DIGITS[int(ch)] for ch in s)
    
    # Match Egyptian phone patterns and general long digit sequences
    return re.sub(r"(\+?[\d٠-٩][\d٠-٩\s().\-]{6,}[\d٠-٩])", repl, text)


def normalize_plain_numbers(text: str) -> str:
    """
    Convert remaining standalone numbers to words.
    Improved to avoid false positives.
    """
    def repl(m):
        return num_token_to_words(m.group(0))

    # Standalone numbers with Arabic digit support
    return re.sub(
        r"(?<![A-Za-zا-ي])(-?[\d٠-٩]+(?:[.,٫][\d٠-٩]+)?)(?![A-Za-zا-ي])",
        repl,
        text
    )


def normalize_abbreviations(text: str) -> str:
    """Expand common Egyptian abbreviations."""
    abbrev_map = {
        r"\bم\.": "متر",
        r"\bكم\.": "كيلومتر",
        r"\bكجم\.": "كيلوجرام",
        r"\bد\.": "دكتور",
        r"\bأ\.د\.": "أستاذ دكتور",
        r"\bش\.": "شارع",
        r"\bص\.ب\.": "صندوق بريد",
    }
    
    for pattern, replacement in abbrev_map.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    
    return text


def normalize_text_for_tts_egyptian(text: str) -> str:
    """
    Enhanced pipeline with better ordering and new features.
    Order matters: specific patterns before general ones.
    """
    if not text or not text.strip():
        return ""
    
    text = text.strip()
    
    # Normalize Arabic digits early
    text = normalize_arabic_digits(text)
    
    # Specific patterns first (most to least specific)
    text = normalize_dates(text)        # Before time (to avoid date slashes)
    text = normalize_time(text)
    text = normalize_percent(text)
    text = normalize_currency(text)
    text = normalize_ranges(text)       # Before plain numbers
    text = normalize_phone_like(text)   # Before plain numbers
    text = normalize_abbreviations(text)
    text = normalize_plain_numbers(text)  # Last for remaining numbers
    
    # Cleanup extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [14]:
from datasets import load_dataset 

dataset = load_dataset("oddadmix/da7ee7_sep_cleaned-8k",split="train")

dataset = dataset.remove_columns(["audio","separated_target_audio", "separated_residual_audio"])
dataset = dataset.rename_column("original_audio", "audio")


Resolving data files:   0%|          | 0/44 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/44 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/34 [00:00<?, ?it/s]

In [8]:
#@title Tokenization Function8

import locale
import torchaudio.transforms as T
import os
import torch
import sys
import numpy as np
sys.path.append('Spark-TTS')
from sparktts.models.audio_tokenizer import BiCodecTokenizer
from sparktts.utils.audio import audio_volume_normalize

audio_tokenizer = BiCodecTokenizer("Spark-TTS-0.5B", "cuda")
def extract_wav2vec2_features( wavs: torch.Tensor) -> torch.Tensor:
        """extract wav2vec2 features"""

        if wavs.shape[0] != 1:

             raise ValueError(f"Expected batch size 1, but got shape {wavs.shape}")
        wav_np = wavs.squeeze(0).cpu().numpy()

        processed = audio_tokenizer.processor(
            wav_np,
            sampling_rate=16000,
            return_tensors="pt",
            padding=True,
        )
        input_values = processed.input_values

        input_values = input_values.to(audio_tokenizer.feature_extractor.device)

        model_output = audio_tokenizer.feature_extractor(
            input_values,
        )


        if model_output.hidden_states is None:
             raise ValueError("Wav2Vec2Model did not return hidden states. Ensure config `output_hidden_states=True`.")

        num_layers = len(model_output.hidden_states)
        required_layers = [11, 14, 16]
        if any(l >= num_layers for l in required_layers):
             raise IndexError(f"Requested hidden state indices {required_layers} out of range for model with {num_layers} layers.")

        feats_mix = (
            model_output.hidden_states[11] + model_output.hidden_states[14] + model_output.hidden_states[16]
        ) / 3

        return feats_mix
def formatting_audio_func(example):
    text = f"{example['source']}: {example['normalized_text']}" if "source" in example else example["normalized_text"]
    text = normalize_text_for_tts_egyptian(text)
    audio_array = example["audio"]["array"]
    sampling_rate = example["audio"]["sampling_rate"]

    target_sr = audio_tokenizer.config['sample_rate']

    if sampling_rate != target_sr:
        resampler = T.Resample(orig_freq=sampling_rate, new_freq=target_sr)
        audio_tensor_temp = torch.from_numpy(audio_array).float()
        audio_array = resampler(audio_tensor_temp).numpy()

    if audio_tokenizer.config["volume_normalize"]:
        audio_array = audio_volume_normalize(audio_array)

    ref_wav_np = audio_tokenizer.get_ref_clip(audio_array)

    audio_tensor = torch.from_numpy(audio_array).unsqueeze(0).float().to(audio_tokenizer.device)
    ref_wav_tensor = torch.from_numpy(ref_wav_np).unsqueeze(0).float().to(audio_tokenizer.device)


    feat = extract_wav2vec2_features(audio_tensor)

    batch = {

        "wav": audio_tensor,
        "ref_wav": ref_wav_tensor,
        "feat": feat.to(audio_tokenizer.device),
    }


    semantic_token_ids, global_token_ids = audio_tokenizer.model.tokenize(batch)

    global_tokens = "".join(
        [f"<|bicodec_global_{i}|>" for i in global_token_ids.squeeze().cpu().numpy()] # Squeeze batch dim
    )
    semantic_tokens = "".join(
        [f"<|bicodec_semantic_{i}|>" for i in semantic_token_ids.squeeze().cpu().numpy()] # Squeeze batch dim
    )

    inputs = [
        "<|task_tts|>",
        "<|start_content|>",
        text,
        "<|end_content|>",
        "<|start_global_token|>",
        global_tokens,
        "<|end_global_token|>",
        "<|start_semantic_token|>",
        semantic_tokens,
        "<|end_semantic_token|>",
        "<|im_end|>"
    ]
    inputs = "".join(inputs)
    return {"text": inputs}


dataset = dataset.map(formatting_audio_func, remove_columns=["audio"])
print("Moving Bicodec model and Wav2Vec2Model to cpu.")
audio_tokenizer.model.cpu()
audio_tokenizer.feature_extractor.cpu()
torch.cuda.empty_cache()

Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb
Moving Bicodec model and Wav2Vec2Model to cpu.


In [17]:
## Push just incase we need to rerun again
dataset.push_to_hub("oddadmix/da7ee7_8k-spark", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/oddadmix/da7ee7_8k-spark/commit/649a656193c31e35c2496f4c9e9b20b78b92884b', commit_message='Upload dataset', commit_description='', oid='649a656193c31e35c2496f4c9e9b20b78b92884b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/oddadmix/da7ee7_8k-spark', endpoint='https://huggingface.co', repo_type='dataset', repo_id='oddadmix/da7ee7_8k-spark'), pr_revision=None, pr_num=None)

In [9]:
from datasets import load_dataset 
dataset = load_dataset("oddadmix/da7ee7_8k-spark", split="train")

README.md:   0%|          | 0.00/365 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/29.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [21]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 6, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = False, # We're doing full float32 s disable mixed precision
        bf16 = False, # We're doing full float32 s disable mixed precision
        logging_steps = 100,
        push_to_hub = True,
        hub_strategy="end",
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "spark-tts-normazlied-da7ee7-8k-lm-egy",
        report_to = "wandb", # Use TrackIO/WandB etc,
    ),
)

In [14]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A6000. Max memory = 47.529 GB.
3.676 GB of memory reserved.


In [22]:
trainer_stats = trainer.train(resume_from_checkpoint=False)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,000 | Num Epochs = 6 | Total steps = 6,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 506,634,112 of 506,634,112 (100.00% trained)


Unsloth: Not an error, but Qwen2Model does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5100,3.169300


KeyboardInterrupt: 

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.


In [25]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4242.6971 seconds used for training.
70.71 minutes used for training.
Peak reserved memory = 15.916 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 33.487 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [26]:
trainer.push_to_hub("oddadmix/spark-tts-normazlied-da7ee7-8k-lm-egy")

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/oddadmix/spark-tts-normazlied-da7ee7-8k-lm-egy/commit/fea43623bd1c9db766994689eb43f828f2f6017e', commit_message='oddadmix/spark-tts-normazlied-da7ee7-8k-lm-egy', commit_description='', oid='fea43623bd1c9db766994689eb43f828f2f6017e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/oddadmix/spark-tts-normazlied-da7ee7-8k-lm-egy', endpoint='https://huggingface.co', repo_type='model', repo_id='oddadmix/spark-tts-normazlied-da7ee7-8k-lm-egy'), pr_revision=None, pr_num=None)

<a name="Inference"></a>
### Inference
Let's run the model! You can change the prompts


In [17]:
input_text =     "الولد الصغير كان بيجري في الشارع وبيضحك بصوت عالي جدًا كأنه لسه كسبان في لعبة"
chosen_voice = None # None for single-speaker

In [20]:
#@title Run Inference

import torch
import re
import numpy as np
from typing import Dict, Any
import torchaudio.transforms as T

# FastModel.for_inference(model) # Enable native 2x faster inference

@torch.inference_mode()
def generate_speech_from_text(
    text: str,
    temperature: float = 0.4,   # Generation temperature
    top_k: int = 50,            # Generation top_k
    top_p: float = 1,        # Generation top_p
    max_new_audio_tokens: int = 1024, # Max tokens for audio part
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
) -> np.ndarray:
    """
    Generates speech audio from text using default voice control parameters.

    Args:
        text (str): The text input to be converted to speech.
        temperature (float): Sampling temperature for generation.
        top_k (int): Top-k sampling parameter.
        top_p (float): Top-p (nucleus) sampling parameter.
        max_new_audio_tokens (int): Max number of new tokens to generate (limits audio length).
        device (torch.device): Device to run inference on.

    Returns:
        np.ndarray: Generated waveform as a NumPy array.
    """

    torch.compiler.reset()

    prompt = "".join([
        "<|task_tts|>",
        "<|start_content|>",
        text,
        "<|end_content|>",
        "<|start_global_token|>"
    ])

    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

    print("Generating token sequence...")
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_audio_tokens, # Limit generation length
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        eos_token_id=tokenizer.eos_token_id, # Stop token
        pad_token_id=tokenizer.pad_token_id # Use models pad token id
    )
    print("Token sequence generated.")


    generated_ids_trimmed = generated_ids[:, model_inputs.input_ids.shape[1]:]


    predicts_text = tokenizer.batch_decode(generated_ids_trimmed, skip_special_tokens=False)[0]
    # print(f"\nGenerated Text (for parsing):\n{predicts_text}\n") # Debugging

    # Extract semantic token IDs using regex
    semantic_matches = re.findall(r"<\|bicodec_semantic_(\d+)\|>", predicts_text)
    if not semantic_matches:
        print("Warning: No semantic tokens found in the generated output.")
        # Handle appropriately - perhaps return silence or raise error
        return np.array([], dtype=np.float32)

    pred_semantic_ids = torch.tensor([int(token) for token in semantic_matches]).long().unsqueeze(0) # Add batch dim

    # Extract global token IDs using regex (assuming controllable mode also generates these)
    global_matches = re.findall(r"<\|bicodec_global_(\d+)\|>", predicts_text)
    if not global_matches:
         print("Warning: No global tokens found in the generated output (controllable mode). Might use defaults or fail.")
         pred_global_ids = torch.zeros((1, 1), dtype=torch.long)
    else:
         pred_global_ids = torch.tensor([int(token) for token in global_matches]).long().unsqueeze(0) # Add batch dim

    pred_global_ids = pred_global_ids.unsqueeze(0) # Shape becomes (1, 1, N_global)

    print(f"Found {pred_semantic_ids.shape[1]} semantic tokens.")
    print(f"Found {pred_global_ids.shape[2]} global tokens.")


    # 5. Detokenize using BiCodecTokenizer
    print("Detokenizing audio tokens...")
    # Ensure audio_tokenizer and its internal model are on the correct device
    audio_tokenizer.device = device
    audio_tokenizer.model.to(device)
    # Squeeze the extra dimension from global tokens as seen in SparkTTS example
    wav_np = audio_tokenizer.detokenize(
        pred_global_ids.to(device).squeeze(0), # Shape (1, N_global)
        pred_semantic_ids.to(device)           # Shape (1, N_semantic)
    )
    print("Detokenization complete.")

    return wav_np

if __name__ == "__main__":
    print(f"Generating speech for: '{input_text}'")
    text = f"{chosen_voice}: " + input_text if chosen_voice else input_text
    generated_waveform = generate_speech_from_text(input_text)

    if generated_waveform.size > 0:
        import soundfile as sf
        output_filename = "generated_speech_controllable.wav"
        sample_rate = audio_tokenizer.config.get("sample_rate", 16000)
        sf.write(output_filename, generated_waveform, sample_rate)
        print(f"Audio saved to {output_filename}")

        # Optional: Play in notebook
        from IPython.display import Audio, display
        display(Audio(generated_waveform, rate=sample_rate))
    else:
        print("Audio generation failed (no tokens found?).")

Generating speech for: 'الولد الصغير كان بيجري في الشارع وبيضحك بصوت عالي جدًا كأنه لسه كسبان في لعبة'
Generating token sequence...
Token sequence generated.
Found 990 semantic tokens.
Found 32 global tokens.
Detokenizing audio tokens...
Detokenization complete.
Audio saved to generated_speech_controllable.wav


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme)
</div>
